In [1]:
import pandas as pd
df = pd.read_csv('covid_data_22.csv')

In [2]:
df.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,NaN,NaN,0,0.0,03/12/2021 03:20:13 PM,Agree,NaN
1,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,NaN,NaN,9,0.0,12/02/2021 02:35:20 PM,Agree,Not agree
2,01/02/2022,AS,11,NaN,NaN,0,0.0,0,NaN,NaN,0,0.0,01/03/2022 03:18:16 PM,NaN,NaN
3,11/22/2021,AL,841461,620483.0,220978.0,703,357.0,16377,12727.0,3650.0,7,3.0,11/22/2021 12:00:00 AM,Agree,Agree
4,05/30/2022,AK,251425,NaN,NaN,0,0.0,1252,NaN,NaN,0,0.0,05/31/2022 01:20:20 PM,NaN,NaN


In [3]:
states = df["state"].unique()
states.sort()
states

array(['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL',
       'FSM', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA',
       'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND',
       'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'NYC', 'OH', 'OK', 'OR', 'PA',
       'PR', 'PW', 'RI', 'RMI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

Based on the visualization table and the list of states above, we want to remove some states for 2 reasons. 

Either 
(1) we don't have census data for it to get relative statistics, or 
(2) NYC was included as its own entity other than New York.
* American Samoa (AS)
* Guam (GU)
* US Virgin Islands (VI)
* Republic of the Marshall Islands (RMI)
* New York City (NYC)
* Federated States of Micronesia (FSM)
* Palau (PW)
* Northern Mariana Islands (MP)

I added all these locations to the helpers.js file to ensure I had found everything to remove.

In [4]:
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [5]:
df_clean = filter_rows_by_values(df, "state", ["AS", "GU", "VI", "RMI", "NYC", "FSM", "PW", "MP"])

In [6]:
df.shape

(60000, 15)

In [7]:
df_clean.shape

(52000, 15)

In [8]:
df_clean.to_csv('covid_data_22_clean.csv', index=False)

In [9]:
df_clean
df_clean['submission_date'] = pd.to_datetime(df_clean['submission_date']).dt.date
df_clean = df_clean.sort_values(by=['submission_date'])

/var/folders/w6/gwwdd5w96ddfwpf2ybyb0wjc0000gn/T/ipykernel_26294/655001797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['submission_date'] = pd.to_datetime(df_clean['submission_date']).dt.date


In [10]:
df_vis = (df_clean.groupby(["submission_date"])
 .agg({'new_case':'sum', 'new_death': 'sum'})
 .reset_index()
 .rename(columns={'submission_date':'date','new_case':'cases', 'new_death':'deaths'}))

In [11]:
# datetime.date(2021, 8, 4)
import datetime

mask = (df_vis['date'] >= datetime.date(2020, 1, 22)) & (df_vis['date'] <= datetime.date(2021, 8, 4))
# mask = (df_vis['date'] >= '2020-01-22') & (df_vis['date'] <= '2021-08-04')
covid_range_df = df_vis.loc[mask]
covid_range_df = covid_range_df.reset_index()


In [12]:
covid_range_df.to_json("covid_cases_deaths.json", orient='records', date_format='iso')

In [13]:
covid_range_df

,index,date,cases,deaths
0,0,2020-01-22,4,0
1,1,2020-01-23,2,0
2,2,2020-01-24,1,0
3,3,2020-01-25,0,0
4,4,2020-01-26,1,0
...,...,...,...,...
556,556,2021-07-31,79796,390
557,557,2021-08-01,60464,363
558,558,2021-08-02,100959,647
559,559,2021-08-03,108659,703
